In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()
%matplotlib inline

Load the latest CSV. This is downloaded by the 'download-from-nomis-api.py' script.

In [ ]:
data = pd.read_csv(
    '../../data/raw/lfs_by_pcon.csv',
    parse_dates=['DATE'],
    usecols=['DATE', 'DATE_NAME', 'GEOGRAPHY_CODE', 'GEOGRAPHY_NAME', 'GEOGRAPHY_TYPE', 'VARIABLE_CODE', 'VARIABLE_NAME', 'MEASURES_NAME', 'OBS_VALUE', 'OBS_STATUS_NAME']
)

Convert the column names to lower case.

In [ ]:
data.columns = data.columns.str.lower()

Tidy up the `variable_name` column

In [ ]:
data.variable_name = data.variable_name.str.strip()

Rename the `obs_value` column to `value`.

In [ ]:
data = data.rename(
    columns={
      'obs_value': 'value',
      'obs_status_name': 'notes'
    }
)

Filter out only the **Variable** measures. The dataset also provides other measures such as **Numerator**, **Denominator** and **Confidence**

In [ ]:
data = data.loc[ data.measures_name == 'Variable' ].drop(columns='measures_name')

Check missing data

In [ ]:
def visualise_coverage():
    plt.imshow(coverage, origin='lower', cmap='viridis', interpolation='nearest', aspect='auto')
    plt.grid(None)
    plt.ylabel('Measure')
    plt.yticks(ticks=range(len(coverage.index)), labels=coverage.index)
    plt.xticks(ticks=range(len(coverage.columns)), labels=coverage.columns.to_series().dt.strftime('%b %Y'), rotation=90, horizontalalignment='center')

counts = pd.DataFrame({
  'available': data.groupby(['date', 'variable_name']).value.count(),
  'total': data.groupby(['date', 'variable_name']).value.size()
})
coverage = pd.Series(
  counts.available / counts.total, name='coverage'
).reset_index().pivot(
  index='variable_name', columns='date', values='coverage'
)

visualise_coverage()

Get the most recently published data per area, to increase coverage

In [ ]:
def get_latest_available(data):
  # Calculate dates
  failsafe_date = data.date.min() - pd.Timedelta("1 days")
  latest_date = data.date.max()
  # Subset the input data
  test_data = data.loc[:, ['geography_code', 'date', 'variable_name', 'value']]
  # Construct a failsafe (to be used in the event that there is no data available)
  # This will have the same index as the most recent values, which we use below...
  failsafe_data = test_data.loc[test_data.date == latest_date]
  failsafe_data.loc[:, ['date', 'value']] = (failsafe_date, -1)
  # Concatenate the test and failsafe_data
  test_data = pd.concat([ test_data, failsafe_data ]).sort_values('date', ascending=True)
  # Get the indexes for the non-zero values
  indexes = test_data.dropna(subset='value').groupby(['geography_code', 'variable_name'])['date'].idxmax()
  return data.loc[indexes]

latest_available = get_latest_available(data)

In [ ]:
latest = data.loc[data.date == data.date.max()].groupby(['variable_name']).value
patched = latest_available.groupby(['variable_name']).value
coverage = pd.DataFrame({
  'latest coverage': latest.count() / latest.size() * 100,
  'patched coverage': patched.count() / patched.size() * 100,
})

coverage

In [ ]:
ax = coverage.plot.barh()

Save the data by parliamentary constituency to a csv

In [ ]:
OUT_FILE = '../../data/processed/labour-market/most_recent_by_pcon_2010.csv'
os.makedirs(os.path.dirname(OUT_FILE), exist_ok=True)
latest_available.loc[ data.geography_type == 'parliamentary constituencies 2010' ].drop(columns=['geography_type']).to_csv(OUT_FILE, index=False)